In [2]:
import pandas as pd 
import pip._vendor.requests as requests
import spotipy
import numpy as np
import matplotlib as plt
from spotipy.oauth2 import SpotifyOAuth
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.multiclass import OutputCodeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression 
from  sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier 
import re
import pickle
scope1 = "user-library-read"
from spotipy.oauth2 import SpotifyClientCredentials
sp =spotipy.Spotify(auth_manager= SpotifyOAuth(client_id = "15fd015d4da742d0bf7da9c6ed3120c7", client_secret = "349c5181636e4e3cbb974b051db425fd", redirect_uri = "http://localhost", scope= scope1))
scope1 = "user-library-read"
from spotipy.oauth2 import SpotifyClientCredentials
sp =spotipy.Spotify(auth_manager= SpotifyOAuth(client_id = "15fd015d4da742d0bf7da9c6ed3120c7", client_secret = "349c5181636e4e3cbb974b051db425fd", redirect_uri = "http://localhost", scope= scope1))

In [ ]:
#gather data using soptify API

output = pd.DataFrame()
target_id = re.search(r'playlist\/(.*)\?', playlistlink).group(1)
name = sp.playlist(playlistlink)["name"]
read = sp.playlist_tracks(playlistlink)
for i in range (len(pd.DataFrame(read))) :
    song = pd.DataFrame(read["items"][i]).reset_index()
    song_uri = song.at[4, "track"]
    analysis = sp.audio_features(song_uri)
    analy = pd.DataFrame(analysis)
    output = pd.concat([output, analy])
output = output.drop(["mode", "duration_ms", "time_signature" , "type"], axis= 1)

In [3]:
output = pd.read_csv("total.csv")

In [6]:
output.loc[output["tag"] == "calm", "tag"] = 0
output.loc[output["tag"] == "happy", "tag"] = 1
output.loc[output["tag"] == "energ", "tag"] = 2
output.loc[output["tag"] == "sad", "tag"] = 3

In [7]:

# Normalize the DataFrame to be between 0 and 1
min_value = output[["Tempo"]].min().min()
max_value = output[["Tempo"]].max().max()
output["Tempo"] = (output[["Tempo"]] - min_value) / (max_value - min_value)

min_value = output[["Loudness"]].min().min()
max_value = output[["Loudness"]].max().max()
output["Loudness"] = (output[["Loudness"]] - min_value) / (max_value - min_value)

min_value = output[["Key"]].min().min()
max_value = output[["Key"]].max().max()
output["Key"] = (output[["Key"]] - min_value) / (max_value - min_value)



*starting to classifed*

In [8]:
#target cloumn
column_select = output[["Danceability", "Energy", "Key", "Loudness", "Speechiness", "Acousticness", "Instrumentalness", "Liveness", "Valence", "Tempo"]]
data_target = output[["tag"]].astype(int)
data_target.dtypes

tag    int64
dtype: object

In [3]:
# load the saved model from file
filename = 'model.pkl'
with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)


In [ ]:
song_x_train, song_x_test, song_y_train, song_y_test = train_test_split(column_select, data_target, test_size= 0.2, random_state= 42)
df_result = []

for i in range(2000):
    model = OneVsRestClassifier(MLPClassifier(hidden_layer_sizes= (16, 10,), activation= "logistic",max_iter= 1000, learning_rate_init= 0.01, alpha = 0.01), n_jobs = -1)
    song_y_train = np.ravel(song_y_train)
    song_y_test = np.ravel(song_y_test)
    model.fit(song_x_train,song_y_train)
    song_y_predict = model.predict(song_x_test)
    output = {"RMSE": np.sqrt(mean_squared_error(song_y_test,song_y_predict)), "accuracy_score":metrics.accuracy_score(song_y_test, song_y_predict)}
    df_result.append(output)
        # print("RMSE", np.sqrt(mean_squared_error(song_y_test,song_y_predict)))
        # print(metrics.accuracy_score(song_y_test, song_y_predict))
        # print("------")
df_analysis = pd.DataFrame(df_result)
df_analysis

In [23]:
len(df_analysis)

2000

In [ ]:
mean = df_analysis["accuracy_score"].mean()
std = df_analysis["accuracy_score"].std()
normal = np.random.normal(mean, std, size = len(df_analysis))
plt.hist(normal, bins = 100,)

In [ ]:
mean = df_analysis["RMSE"].mean()
std = df_analysis["RMSE"].std()
normal = np.random.normal(mean, std, size = len(df_analysis))
plt.hist(normal, bins = 100, density= True)